In [1]:
import sys
import os
import pandas as pd
import logging
from tqdm import tqdm
import statsmodels.api as sm
import datetime
import sqlite3
from scipy import stats
from statsmodels.stats.diagnostic import het_white

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

sys.path.insert(0, r"E:/Thesis/crypto_project")
from factor_model.model_update.loaders.crypto_coin_loaders import (
    get_extended_ticker_list,
)
from factor_model.model_update.loaders.price_data_loaders import (
    generate_ytickets,
    generate_price_data_map,
)
from factor_model.model_update.loaders.market_cap_loader import (
    generate_market_cap_only,
)
from factor_model.model_update.loaders.crypto_coin_loaders import (
    get_extended_ticker_list,
)
from statsmodels.stats.outliers_influence import variance_inflation_factor

from factor_model.model_update.return_generators.total_return_calculation import (
    generate_return_data,
)
from datetime import date, timedelta
from factor_model.model_update.database_generators.generate_databases import (
    refresh_database,
)
from factor_model.model_update.estimation_universe.estimation_core_universe import (
    generate_estimation_basis,
)
from factor_model.model_update.styles.return_based import (
    generate_x_month_price_change,
)
from factor_model.model_update.styles.volume_based import (
    generate_x_month_aggregate_volume,
)
from factor_model.model_update.factor_return_estimation.factor_return_estimation import (
    create_factor_return_data,
)
from factor_model.model_update.loaders.risk_free_rate_loader import (
    get_risk_free_rate_history,
)
from factor_model.model_update.return_generators.excess_return_calculation import (
    generate_excess_returns,
)
from factor_model.risk_calculations.core_universe_portfolio import (
    generate_market_portfolio,
)
from factor_model.risk_calculations.specific_risk import (
    generate_raw_portfolio_specific_risk,
)


DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    RAW_DATA_DB,
    RETURN_DB,
    SPECIFIC_RISK,
    FACTOR_MODEL_ESTIMATES,
    FIX_SET_OF_HALF_LIFES,
)
from factor_model.model_update import UPDATE_PARAMS, RISK_CALCULATION_PARAMETERS

In [2]:
parameters = UPDATE_PARAMS.copy()
parameters["CORE_UNIV_START"] = parameters["ESTIMATION_DAY"] - timedelta(
    parameters["PRESENT_IN_MARKET"]
)
parameters["GENERATE_DATABASE"] = True
parameters["HORIZON"] = "1y"  # 1y is also fair
parameters["NOBS"] = 3000
parameters["MARKET_CAP_COVERAGE"] = 0.7

In [3]:
tickers = get_extended_ticker_list(
    parameters["NOBS"]
)  # the number of crypto can be adjusted...
# TODO : survival bias should be fixed by adding discontiued crypto currency
tickers = [f for f in tickers if f.endswith("USD")]
tickers = sorted(list(set(tickers)))

In [4]:
yfinance_tickers = generate_ytickets(tickers)
price_data_map, tickers_with_missing_data = generate_price_data_map(
    yfinance_tickers, parameters["HORIZON"]
)

problematic_keys = []
for key in price_data_map.keys():
    if len(price_data_map[key].columns) > 8:
        problematic_keys.append(key)

for key in problematic_keys:
    del price_data_map[key]

# refresh tickers and throw out those ones, which have no price history
drop_keys = list()
for key in price_data_map.keys():
    if len(price_data_map[key]) < 2:  # minimum number of obs for returns are 2
        del yfinance_tickers[key]
        drop_keys.append(key)
for key in drop_keys:
    del price_data_map[key]

2024-11-26 15:52:35,171 - ERROR - AIC-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
2024-11-26 15:52:35,174 - WARNING - No data for AIC-USD!
2024-11-26 15:52:39,097 - ERROR - AMC31871-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-26 15:52:39,099 - WARNING - No data for AMC31871-USD!
2024-11-26 15:52:40,753 - ERROR - ANYONE-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-26 15:52:40,756 - WARNING - No data for ANYONE-USD!
2024-11-26 15:52:44,176 - ERROR - ARTFI-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-26 15:52:44,179 - WARNING - No data for ARTFI-USD!
2024-11-26 15:52:45,831 - ERROR - ATH30083-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-26 15:52:45,835 - WARNING - No data for ATH30083-USD!
2024-11-26 15:52:48,791 - ERROR - AXLFRXET

In [5]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RAW_DATA_DB)) as conn:
    max_dates = pd.read_sql(
        "SELECT MAX(date) as date, symbol FROM raw_price_data group by symbol", conn
    )
max_dates_struct = max_dates.set_index("symbol").to_dict()
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RAW_DATA_DB)) as conn:
    max_id = pd.read_sql("SELECT MAX(id) as id FROM raw_price_data", conn)
idx_index = int(max_id["id"].values[0]) + 1

In [6]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RAW_DATA_DB)) as conn:
    for key in price_data_map:
        latest_date = max_dates_struct["date"].get(key, "2008-01-01")

        df_temp = price_data_map[key]
        df_temp = df_temp[df_temp["date"].astype(str) > latest_date].copy()
        df_temp["symbol"] = key
        df_temp.columns = [col.lower().replace(" ", "_") for col in df_temp.columns]
        df_temp.reset_index(inplace=True)
        df_temp.rename(columns={"index": "id"}, inplace=True)
        df_temp["id"] = range(idx_index, len(df_temp) + idx_index)
        idx_index += len(df_temp)
        df_temp.to_sql("raw_price_data", conn, if_exists="append", index=False)

In [7]:
# market_cap_df_full = generate_market_cap_data(yfinance_tickers)
market_cap_df = generate_market_cap_only(yfinance_tickers)
# generate square root of cap weighting
market_cap_df["transformed_market_cap"] = parameters["WEIGHT_FUNCTION"](
    market_cap_df["market_cap"]
)
market_cap_df.sort_values(by="transformed_market_cap", ascending=False, inplace=True)
total_return_data_map = generate_return_data(price_data_map)

2024-11-26 16:03:23,929 - INFO - No data for ticker: ABC24497-USD
2024-11-26 16:03:26,225 - INFO - No data for ticker: ABO-USD
2024-11-26 16:06:55,482 - INFO - No data for ticker: BABY21709-USD
2024-11-26 16:07:13,877 - INFO - No data for ticker: BANANAS-USD
2024-11-26 16:07:50,349 - INFO - No data for ticker: BEFX23583-USD
2024-11-26 16:11:38,271 - INFO - No data for ticker: CHAD30937-USD
2024-11-26 16:14:18,303 - INFO - No data for ticker: DCI-USD
2024-11-26 16:15:36,034 - INFO - No data for ticker: DOGE28438-USD
2024-11-26 16:17:40,460 - INFO - No data for ticker: ETHEREUM27840-USD
2024-11-26 16:18:31,659 - INFO - No data for ticker: FEES-USD
2024-11-26 16:18:40,837 - INFO - No data for ticker: FILST-USD
2024-11-26 16:19:11,567 - INFO - No data for ticker: FOMO30794-USD
2024-11-26 16:20:13,856 - INFO - No data for ticker: GAS16606-USD
2024-11-26 16:20:38,795 - INFO - No data for ticker: GGR-USD
2024-11-26 16:22:53,290 - INFO - No data for ticker: HNY-USD
2024-11-26 16:25:32,941 - IN

In [8]:
estimation_dates = list(
    total_return_data_map["BTC-USD"].tail(parameters["ESTIMATION_HORIZON"])["date"]
)
estimation_dates = list(
    total_return_data_map["ETC-USD"].tail(parameters["ESTIMATION_HORIZON"])["date"]
)

with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    max_fac_return_date = pd.read_sql(
        "SELECT MAX(date) as date FROM factor_returns", conn
    )["date"].values[0]
last_estima_date = date(
    int(max_fac_return_date[0:4]),
    int(max_fac_return_date[5:7]),
    int(max_fac_return_date[8:]),
)
estimation_dates = [
    date_value
    for date_value in estimation_dates
    if date_value > datetime.date(2018, 5, 8) and date_value > last_estima_date
]  # six months after ETH-USD has data in yfinance

In [9]:
estimation_dates

[datetime.date(2024, 11, 22),
 datetime.date(2024, 11, 23),
 datetime.date(2024, 11, 24),
 datetime.date(2024, 11, 25),
 datetime.date(2024, 11, 26)]

In [10]:
risk_free_rate_data = get_risk_free_rate_history(estimation_dates)
excess_return_data_map = generate_excess_returns(
    total_return_data_map, risk_free_rate_data
)

[*********************100%***********************]  1 of 1 completed
2024-11-26 16:48:42,241 - INFO - First SOFR date is 2024-11-12 00:00:00, before this date, the 3M US Treasury rate is used as risk free rate.
2024-11-26 16:48:42,247 - INFO - 3M US Treasury rate is used for 4995 trading days.
E:\Thesis/crypto_project\factor_model\model_update\loaders\risk_free_rate_loader.py:63: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ].fillna(method="backfill")
E:\Thesis/crypto_project\factor_model\model_update\loaders\risk_free_rate_loader.py:77: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  treasury_data["risk_free_rate"] = treasury_data["risk_free_rate"].fillna(
E:\Thesis/crypto_project\factor_model\model_update\loaders\risk_free_rate_loader.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame 

In [11]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RETURN_DB)) as conn:
    max_dates = pd.read_sql(
        "SELECT MAX(date) as date, symbol FROM returns group by symbol", conn
    )
max_dates_struct = max_dates.set_index("symbol").to_dict()

In [12]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RETURN_DB)) as conn:
    for key in excess_return_data_map:
        latest_date = max_dates_struct["date"].get(key, "2008-01-01")
        df_temp = excess_return_data_map[key]
        df_temp = df_temp[df_temp["date"].astype(str) > latest_date].copy()
        df_temp["symbol"] = key
        df_temp.columns = [col.lower().replace(" ", "_") for col in df_temp.columns]
        df_temp.reset_index(inplace=True)
        df_temp.rename(columns={"index": "id"}, inplace=True)
        df_temp["id"] = range(idx_index, len(df_temp) + idx_index)
        idx_index += len(df_temp)
        df_temp.to_sql("returns", conn, if_exists="append", index=False)

In [13]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RETURN_DB)) as conn:
    max_date_risk_free = pd.read_sql(
        "SELECT MAX(date) as date FROM risk_free_rates", conn
    )["date"].values[0]
    last_estima_date = date(
        int(max_date_risk_free[0:4]),
        int(max_date_risk_free[5:7]),
        int(max_date_risk_free[8:]),
    )
    rft_sel = risk_free_rate_data[risk_free_rate_data["date"] > last_estima_date].copy()
    rft_sel["id"] = range(0, len(rft_sel))
    rft_sel.to_sql("risk_free_rates", conn, if_exists="append", index=False)

In [14]:
momentum_move_map = generate_x_month_price_change(
    price_data_map, x_len=6, month_len=parameters["MONTH_LENGTH"]
)
reversal_map = generate_x_month_price_change(
    price_data_map, x_len=1, month_len=parameters["MONTH_LENGTH"]
)
volume_map = generate_x_month_aggregate_volume(
    price_data_map, x_len=1, month_len=parameters["MONTH_LENGTH"]
)
daily_data_maps = {}
daily_data_maps["reversal"] = reversal_map
daily_data_maps["momentum"] = momentum_move_map
daily_data_maps["return"] = excess_return_data_map
daily_data_maps["volume"] = volume_map

E:\Thesis/crypto_project\factor_model\model_update\styles\return_based.py:21: RuntimeWarning: divide by zero encountered in divide
  close_price_changes = np.matrix(
E:\Thesis/crypto_project\factor_model\model_update\styles\return_based.py:21: RuntimeWarning: invalid value encountered in divide
  close_price_changes = np.matrix(


In [15]:
for key in excess_return_data_map.keys():
    try:
        v = min(excess_return_data_map[key]["date"])
    except Exception:
        print(key)

BCZERO-USD


In [18]:
tstats_all = list()
coefficients_all = list()
vif_all = list()
model_summary = list()

for date in tqdm(estimation_dates):
    # step 0 : assemble and save estimation data
    estimation_basis, _ = generate_estimation_basis(
        excess_return_data_map, market_cap_df, date, parameters
    )
    factor_return_data = create_factor_return_data(
        estimation_basis, parameters, date, daily_data_maps
    )
    factor_return_data_core = factor_return_data[
        factor_return_data["core_universe"] > 0
    ]
    if parameters["GENERATE_DATABASE"]:
        refresh_database(
            symbol_level_data={
                str(date): factor_return_data.rename(columns={"ticker": "symbol"})
            },
            database_location=DATABASE_LOCATION,
            database_name=FACTOR_MODEL_ESTIMATES,
            database_table_name="exposures",
            delete_database=False,
            key_field_name="date",
            update_mode="append",
        )
    # step 1 estimation
    try:
        mod_wls = sm.WLS(
            endog=factor_return_data_core["return"],
            exog=factor_return_data_core[parameters["REGRESSORS_SET1"]],
            weights=factor_return_data_core["transformed_market_cap"],
        ).fit()

        # step 2 save results
        coefficient_date = mod_wls.params.to_frame().T
        coefficient_date["date"] = date
        coefficients_all.append(coefficient_date)

        tstat_date = mod_wls.tvalues.to_frame().T
        tstat_date["date"] = date
        tstats_all.append(tstat_date)

        # step 2 b, additional tests...
        white_test = het_white(mod_wls.resid,  mod_wls.model.exog)
        res_jb = stats.jarque_bera(mod_wls.resid)

        model_summary.append(
            pd.DataFrame(
                {
                    "date": [date],
                    "r2_core": [mod_wls.rsquared],
                    "r2_adj": [mod_wls.rsquared_adj],
                    "nobs": [mod_wls.nobs],
                    "white_lm_pvalue": [float(white_test[1])],
                    "jb_pvalue": [float(res_jb.pvalue)]
                }
            )
        )

        exog = factor_return_data[
            parameters["REGRESSORS_SET1"] + parameters["REGRESSORS_SET2"]
        ].to_numpy()
        num_features = exog.shape[1]
        vif_values = [variance_inflation_factor(exog, i) for i in range(num_features)]
        vif_result_date = (
            pd.Series(
                vif_values,
                index=parameters["REGRESSORS_SET1"] + parameters["REGRESSORS_SET2"],
            )
            .to_frame()
            .T
        )
        vif_result_date["date"] = date
        vif_all.append(vif_result_date)

        specific_returns = factor_return_data[["ticker"]].copy()
        specific_returns["specific_return"] = factor_return_data[
            "return"
        ] - mod_wls.predict(factor_return_data[parameters["REGRESSORS_SET1"]])
        if parameters["GENERATE_DATABASE"]:
            refresh_database(
                symbol_level_data={
                    str(date): specific_returns.rename(columns={"ticker": "symbol"})
                },
                database_location=DATABASE_LOCATION,
                database_name=FACTOR_MODEL_ESTIMATES,
                database_table_name="specific_returns",
                delete_database=False,
                key_field_name="date",
                update_mode="append",
            )
    except Exception:
        print(date)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:51<00:00, 10.33s/it]


In [19]:
all_tstat = pd.concat(tstats_all, axis=0)
all_coeff = pd.concat(coefficients_all, axis=0)
all_vif = pd.concat(vif_all, axis=0)
all_model_summary = pd.concat(model_summary, axis=0)
if parameters["GENERATE_DATABASE"]:
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_tstat},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="t_statistics",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_coeff},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="factor_returns",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_vif},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="vifs",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_model_summary},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="r_squares",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )

In [20]:
# specific risk related

In [21]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    full_specific_returns = pd.read_sql_query(f"SELECT * FROM specific_returns", conn)

with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    estim_universe = pd.read_sql_query(
        f"SELECT symbol, transformed_market_cap, date, core_universe FROM exposures where core_universe > 0",
        conn,
    )

In [22]:
full_specific_returns = full_specific_returns[
    full_specific_returns["symbol"].isin(
        list(set(estim_universe[estim_universe["core_universe"] > 0]["symbol"]))
    )
].copy()


In [23]:
dates = [
    date_v
    for date_v in list(full_specific_returns["date"].unique())[1:]
    if date_v in [str(d) for d in estimation_dates]
]
risk_calculation_parameters = RISK_CALCULATION_PARAMETERS.copy()

In [24]:
def multi_process_calc(temp_half_life):
    risk_calculation_parameters = {"specific_risk_half_life": temp_half_life}
    risk_for_half_life = full_specific_returns.groupby("symbol").apply(
        lambda group: pd.DataFrame({
            'date': group['date'],
            'specific_risk': group['specific_return']
                .ewm(halflife=risk_calculation_parameters['specific_risk_half_life'])
                .std(),
            'symbol': group['symbol'].iloc[0]
        })
    )
    return risk_for_half_life.reset_index(drop=True)

In [25]:
all_core_spec_risk = {}

for temp_half_life in tqdm(FIX_SET_OF_HALF_LIFES):
    all_core_spec_risk[temp_half_life] = multi_process_calc(temp_half_life)
    all_core_spec_risk[temp_half_life].set_index(["date", "symbol"], inplace=True)

  0%|          | 0/38 [00:00<?, ?it/s]C:\Users\Bebesi Laci\AppData\Local\Temp\ipykernel_22716\4244655963.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  risk_for_half_life = full_specific_returns.groupby("symbol").apply(
  3%|▎         | 1/38 [00:10<06:16, 10.18s/it]C:\Users\Bebesi Laci\AppData\Local\Temp\ipykernel_22716\4244655963.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  risk_for_half_life =

In [26]:
estim_universe_dict = {date: df for date, df in estim_universe.groupby("date")}
for key in all_core_spec_risk.keys():
    all_core_spec_risk[key] = {
        date: df for date, df in all_core_spec_risk[key].groupby("date")
    }

In [27]:
all_combined_specific_risk_estimates = list()

for date in tqdm(dates):
    # 1. gather exposures
    exposure = estim_universe_dict[date]
    # 2. identify universe_info
    market_portfolio = generate_market_portfolio(exposure)

    for half_life in FIX_SET_OF_HALF_LIFES:
        # 3. grab relevant spec risks
        tempd_df = all_core_spec_risk[half_life][date]
        tempd_df.reset_index(inplace=True)
        tempd_df.set_index("symbol", inplace=True)
        tempd_df.dropna(inplace=True)

        all_combined_specific_risk_estimates.append(
            pd.DataFrame(
                {
                    "date": [date],
                    "half_life": [half_life],
                    "specific_risk": generate_raw_portfolio_specific_risk(
                        tempd_df.to_dict()["specific_risk"], market_portfolio
                    ),
                }
            )
        )

100%|██████████| 5/5 [00:01<00:00,  3.20it/s]


In [28]:
combined_spec_risk = pd.concat(all_combined_specific_risk_estimates)


In [29]:
refresh_database(
    symbol_level_data={str(date): combined_spec_risk},
    database_location=DATABASE_LOCATION,
    database_name=SPECIFIC_RISK,
    database_table_name="core_specific_risk",
    delete_database=False,
    key_field_name="version_date",
    update_mode="append",
)

In [30]:
all_model_summary

,date,r2_core,r2_adj,nobs,white_lm_pvalue,jb_pvalue
0,2024-11-22,0.343139,0.332875,326.0,1.168468e-08,0.0
0,2024-11-23,0.161575,0.148433,325.0,5.374121e-24,0.0
0,2024-11-24,0.098544,0.084458,326.0,4.774873e-01,0.0
0,2024-11-25,0.211982,0.199631,325.0,5.841967e-01,0.0
0,2024-11-26,0.124864,0.111147,325.0,4.982262e-51,0.0


In [31]:
all_tstat

,market,size,momentum,reversal,volume,new_coin,date
0,9.839036,-0.453970,-3.400910,11.240380,-2.222096,-3.152094,2024-11-22
0,9.571230,-3.187019,-1.581365,5.499009,-0.150742,-1.114799,2024-11-23
0,5.497462,-4.435293,-2.154122,0.460083,2.217174,-1.040063,2024-11-24
0,-10.233831,-0.070870,-3.126241,-5.381333,-1.081239,-0.010378,2024-11-25
0,-4.871289,-2.226906,3.265370,2.922187,2.898257,-0.571885,2024-11-26


In [32]:
all_vif

,market,size,momentum,reversal,volume,new_coin,date
0,38.202106,1.049308,1.264557,1.239163,1.022793,1.004118,2024-11-22
0,38.297616,1.052006,1.294390,1.267575,1.023053,1.003558,2024-11-23
0,38.425924,1.057052,1.298463,1.279369,1.023671,1.003647,2024-11-24
0,38.437476,1.062573,1.290295,1.275931,1.024227,1.003610,2024-11-25
0,37.354840,1.064861,1.347363,1.338683,1.024936,1.004341,2024-11-26


In [33]:
all_coeff

,market,size,momentum,reversal,volume,new_coin,date
0,0.033034,-0.002728,-0.013522,0.045122,-0.012936,-0.011066,2024-11-22
0,0.040984,-0.024432,-0.008136,0.028558,-0.001124,-0.004984,2024-11-23
0,0.017389,-0.025009,-0.008859,0.001905,0.012262,-0.003443,2024-11-24
0,-0.030557,-0.000378,-0.012030,-0.021830,-0.005672,-0.000032,2024-11-25
0,-0.013286,-0.010838,0.010577,0.009541,0.013853,-0.001625,2024-11-26
